In [1]:
import cv2
import numpy as np 
import matplotlib as plt
    

In [2]:
def click_callback(event,x,y,flags,params):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(x,y)
    return x,y

In [3]:
ref = 'reference.png'
cam0 = 'cam0.jpeg'
cam1 = 'cam1.jpeg'
cam2 = 'cam2.jpeg'
switch = 0
if switch == 0:
    _file = ref
elif switch == 1:
    _file = cam0
elif switch == 2:
    _file = cam1
else:
    _file = cam2
img = cv2.imread(_file)
img = cv2.resize(img,(480,270))
cv2.imshow('cam',img)
cv2.setMouseCallback('cam',click_callback)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
width = 600
height = 300

In [3]:
ref_points = []
cam0_points = []
with open('ref_cv2.txt','r') as f:
    for lines in f.readlines():
        pointx,pointy = int(lines.split(" ")[0]),int(lines.split(" ")[1])
        ref_points.append([pointx,pointy])
    f.close()
with open('cam1_cv2.txt','r') as f:
    for lines in f.readlines():
        pointx,pointy = int(lines.split(" ")[0]),int(lines.split(" ")[1])
        cam0_points.append([pointx,pointy])
    f.close()

In [4]:
img = cv2.imread('cam1.jpeg')
cam0_points = np.float32(cam0_points)
ref_points  = np.float32(ref_points)
M,_ = cv2.findHomography(ref_points,cam0_points)
dst = cv2.warpPerspective(img,M,(width,height))
cv2.imshow('frame',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
import json
with open('camera0.txt','r') as f:
    data = json.load(f)
    f.close()
points_per_frame = []
for frame in data:
    temp = []
    for frame_data in frame:
        dic = {}
        dic['class'] = frame_data['class']
        dic['pointx'] = frame_data['start_X'] + (frame_data['width']/2)
        dic['pointy'] = frame_data['start_Y'] + (frame_data['height']/2)
        temp.append(dic)
    points_per_frame.append(temp)

import json
with open("points.txt",'w') as f:
    json.dump(points_per_frame,f)
f.close()

In [7]:
def warp_points(point_data,M):
    points_to_draw = []
    for point in point_data:
        mat = np.array([point['pointx'],point['pointy'],1])
        mat = np.sum(mat*M,axis = 1)
        mat = mat/mat[2] #normalize by z axis
        color = (255,0,0) if point['class'] == 'standing' else (0,0,255)
        points_to_draw.append([int(mat[0]),int(mat[1]),color])
    return points_to_draw

fourcc = cv2.VideoWriter_fourcc(*'XVID')
video=cv2.VideoWriter('cam2_top.avi',fourcc, 30, (width,height))        
capture = cv2.VideoCapture('cam1.avi')
frames = 0
while True:
    ret,frame = capture.read()
    if not ret:
        break
    frame = cv2.warpPerspective(frame,M,(width,height))
    #points = warp_points(points_per_frame[frames],M)
    # for point in points:
    #     cv2.circle(frame,(point[0],point[1]),2,point[2],2)
    cv2.imshow('frame',frame)
    video.write(frame)
    if cv2.waitKey(33) & 0xff == ord('q'):
        break
    frames+=1

capture.release()
video.release()
cv2.destroyAllWindows()